# RNA simulation from OpenMM force field

This notebook is YELLOW because:
- Sage has not been validated to work with Amber force fields and has a different partial charge philosophy
- Many of the functions used are experimental

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1

## Load PDB

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3158920

In [ ]:
def molecule_from_rna_sequence(sequence):
    import random

    from openff.toolkit import Molecule
    from rdkit import Chem
    from rdkit.Chem import AllChem

    rna = Chem.MolFromSequence(sequence, flavor=2)

    for atom in rna.GetAtoms():
        if atom.GetNumImplicitHs() == 1 and [
            a.GetSymbol() for a in atom.GetNeighbors()
        ] == ["P"]:
            atom.SetFormalCharge(-1)

    Chem.SanitizeMol(rna)
    offmol = Molecule.from_rdkit(
        rna,
        allow_undefined_stereo=True,
    )

    return offmol


ribozyme = molecule_from_rna_sequence("GUGGC")
rna_substrate = molecule_from_rna_sequence("GCCU")

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

# PheAMP, phenylalanine adenosine monophosphate
pheamp = Molecule.from_smiles(
    "c1ccccc1C[C@H](N)C(=O)O[P@]([O-])(=O)OC[C@@H]2[C@@H](O)[C@@H](O)[C@@H](O2)N3C4N=CN=C(N)C=4N=C3",
    allow_undefined_stereo=False,
)
pheamp.visualize("rdkit", show_all_hydrogens=False)

In [ ]:
from openff.toolkit import Topology

top = Topology.from_pdb(
    "inputs/ribozymes.pdb",
    unique_molecules=[pheamp, ribozyme, rna_substrate],
)
top.visualize()

## Split topologies

In [ ]:
molecules = list(top.molecules)

rna_top = Topology.from_molecules(molecules[:2])
rna_top.box_vectors = top.box_vectors

sage_top = Topology.from_molecules(molecules[2:])
sage_top.box_vectors = top.box_vectors

In [ ]:
sage_top.visualize()

## Parametrize

In [ ]:
from openff.toolkit import ForceField

sage = ForceField("openff-2.2.0.offxml")
sage_interchange = sage.create_interchange(sage_top)

In [ ]:
from tempfile import NamedTemporaryFile

import numpy as np
import openmm.app
from openff.interchange import Interchange
from openff.toolkit import Molecule, Topology
from rdkit import Chem

openmm_force_field=openmm.app.ForceField(
    # "amber14/protein.ff14SB.xml",
    "amber14/RNA.OL3.xml",
    # "amber14/DNA.OL15.xml",
    # "amber14/lipid17.xml",
    # "amber14/GLYCAM_06j-1.xml",
    # "amber14/tip3p.xml",
)

openmm_system = openmm_force_field.createSystem(
    rna_top.to_openmm(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    switchDistance=8 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

# Create the Interchange
rna_interchange = Interchange.from_openmm(
    topology=rna_top,
    system=openmm_system,
    positions=rna_top.get_positions(),
)

rna_interchange.visualize()

## Combine Interchanges

In [ ]:
# Work around a bug in Interchange 0.3.26
rna_interchange['Electrostatics'].cutoff = rna_interchange['vdW'].cutoff

solvated_interchange = sage_interchange.combine(rna_interchange)

In [ ]:
solvated_interchange.visualize()

## Simulate!

In [ ]:
simulation = solvated_interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtosecond,
    )
)

simulation.minimizeEnergy()

# Add a reporter to record the structure every few steps
dcd_reporter = openmm.app.DCDReporter(file="rna.dcd", reportInterval=1000)
simulation.reporters.append(dcd_reporter)

simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
from utils import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "rna.dcd")
w.add_licorice(selection="not (HOH NA CL)")
w.add_spacefill(selection="HOH", opacity=0.05)
w.add_spacefill(selection="NA CL")
w.add_unitcell()
w